# Redes Neurais Convolucionais

Vamos agora implementar a rede [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das redes que trouxeram todo esse interesse para a área de *deep learning*.





## Configuração do ambiente

In [ ]:
import time, os, sys, numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torch import optim
from torchsummary import summary

import time, os, sys, numpy as np

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids= list(range(n))

## Carregamento da base da dados

A função `load_data_cifar10` carrega e prepara o dataset CIFAR-10 para treinamento e teste.

CIFAR-10 é um conjunto de dados amplamente utilizado em tarefas de visão computacional e aprendizado de máquina, especialmente em problemas de classificação de imagens. Ele consiste em imagens coloridas de dimensão 32 x 32, separadas em 10 classes.

In [ ]:
def load_data_cifar10(batch_size, resize=None, root=os.path.join('~', '.pytorch', 'datasets', 'cifar10')):
    root = os.path.expanduser(root)

    transformer = []
    if resize:
        transformer += [torchvision.transforms.Resize(resize)]
    transformer += [torchvision.transforms.ToTensor()]
    transformer = torchvision.transforms.Compose(transformer)

    cifar10_train = torchvision.datasets.CIFAR10(root=root, train=True, download=True, transform=transformer)
    cifar10_test = torchvision.datasets.CIFAR10(root=root, train=False, download=True, transform=transformer)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = torch.utils.data.DataLoader(cifar10_train,
                                            batch_size, shuffle=True,
                                            num_workers=num_workers)

    test_iter = torch.utils.data.DataLoader(cifar10_test,
                                            batch_size, shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter

## Funções auxiliares

* `evaluate_accuracy` calcula a acurácia de um modelo em um dataset.

* `train_validate` implemneta o treinamento e validação de uma rede.

In [ ]:
def evaluate_accuracy(data_iter, net, loss):
    acc_sum, n, l = torch.Tensor([0]), 0, 0
    net.eval()

    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y).sum()
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
          n += y.size()[0]

    return acc_sum.item() / n, l.item() / len(data_iter)

def train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs):
    print('training on', device)

    for epoch in range(num_epochs):
        net.train()
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            y_hat = net(X)

            trainer.zero_grad()
            l = loss(y_hat, y).sum()

            l.backward()
            trainer.step()

            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size()[0]

        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)

        print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))

## AlexNet

Agora já temos todo o conhecimento necessário para implementar nossa primeira arquitetura moderna.

Vamos implementar a [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), uma das arquiteturas mais famosas dessa nova onda de rede neurais.

<p align="center">
  <img width=700 src="https://www.researchgate.net/profile/Jaime_Gallego2/publication/318168077/figure/fig1/AS:578190894927872@1514862859810/AlexNet-CNN-architecture-layers.png">
</p>

<p align="center">
  <img width=700 src="https://miro.medium.com/max/700/1*vXBvV_Unz3JAxytc5iSeoQ.png">
</p>

### Atividade

Implementa sua rede neural baseada na AlexNet.

Lembre-se que, após cada camada de convolução e linear, há uma ativação não linear ReLU.

**Dica:** utilize blocos *sequential* para diminuir a complexidade da função de forward.

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, input_channels, classes=10, **kwargs):
        super(AlexNet, self).__init__(**kwargs)

        self.conv1 = nn.Sequential(
            nn.Conv2d(input_channels, 96, kernel_size=11, stride=4, padding=0),
            nn.ReLU(),
        )

        self.maxpool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.conv2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
        )

        self.maxpool2 = nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.conv3 = nn.Sequential(
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
        )

        self.maxpool5 = nn.MaxPool2d(kernel_size = 3, stride = 2)

        self.flat = nn.Flatten()

        self.fc6 = nn.Sequential(
            nn.Linear(9216, 4096),
            nn.ReLU()
        )

        self.fc7 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU()
        )

        self.fc8 = nn.Linear(4096, classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.maxpool5(x)
        x = self.flat(x)
        x = self.fc6(x)
        x = self.fc7(x)
        out = self.fc8(x)

        return out

In [ ]:
num_epochs, lr, batch_size, weight_decay = 20, 0.001, 100, 0.0001

net = AlexNet(input_channels=3, classes=10)
net.to(device)
print(summary(net,(3,227,227)))

loss = nn.CrossEntropyLoss()

train_iter, test_iter = load_data_cifar10(batch_size, resize=227)

trainer = optim.Adam(net.parameters(), lr=lr, weight_decay=weight_decay)

train_validate(net, train_iter, test_iter, batch_size, trainer, loss, num_epochs)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          34,944
              ReLU-2           [-1, 96, 55, 55]               0
         MaxPool2d-3           [-1, 96, 27, 27]               0
            Conv2d-4          [-1, 256, 27, 27]         614,656
              ReLU-5          [-1, 256, 27, 27]               0
         MaxPool2d-6          [-1, 256, 13, 13]               0
            Conv2d-7          [-1, 384, 13, 13]         885,120
              ReLU-8          [-1, 384, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]       1,327,488
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
        MaxPool2d-13            [-1, 256, 6, 6]               0
          Flatten-14                 [-

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /root/.pytorch/datasets/cifar10/cifar-10-python.tar.gz to /root/.pytorch/datasets/cifar10
Files already downloaded and verified
training on cuda


/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1, train loss 1.8529, train acc 0.310, test loss 1.5753, test acc 0.427, time 107.1 sec
epoch 2, train loss 1.3988, train acc 0.492, test loss 1.2647, test acc 0.545, time 104.6 sec
epoch 3, train loss 1.1834, train acc 0.579, test loss 1.0976, test acc 0.609, time 103.7 sec
epoch 4, train loss 1.0185, train acc 0.639, test loss 1.0133, test acc 0.648, time 105.3 sec
epoch 5, train loss 0.8846, train acc 0.688, test loss 0.9714, test acc 0.664, time 103.8 sec
epoch 6, train loss 0.7719, train acc 0.728, test loss 0.8920, test acc 0.693, time 102.8 sec
epoch 7, train loss 0.6790, train acc 0.762, test loss 0.9016, test acc 0.694, time 105.4 sec
epoch 8, train loss 0.5863, train acc 0.795, test loss 0.9019, test acc 0.695, time 103.8 sec
epoch 9, train loss 0.5033, train acc 0.823, test loss 0.9189, test acc 0.708, time 104.5 sec
epoch 10, train loss 0.4177, train acc 0.852, test loss 1.0099, test acc 0.702, time 103.1 sec
epoch 11, train loss 0.3408, train acc 0.880, test loss 0.9